# Feedback Controls
### Tutorial on Feedback Systems (part 1 of N)
##### R. X Adhikari (6/2020)

### Useful References
* [Feedback Controls by Astrom and Murray](http://www.cds.caltech.edu/~murray/amwiki/index.php/Second_Edition)
* [Why Null Measurements](https://courses.lumenlearning.com/physics/chapter/21-5-null-measurements/)
* [Gravitational Wave Detectors, Saulson (2017)](https://www.google.com/books/edition/_/5QVGDQEACAAJ?hl=en&sa=X&ved=2ahUKEwicmp_3v5TqAhUUJTQIHTdSDEoQre8FMAB6BAgAEA4)
* [Controls Tutorials](https://wiki.ligo.org/CSWG/Training_References) from the LIGO/Virgo/KAGRA Control Systems Working Group
* [State Space Methods](https://www.google.com/books/edition/Control_System_Design/2M1SAAAAMAAJ?hl=en)

In [ ]:
import sys,os
import numpy as np

from scipy import signal as sig
#from scipy import interpolate
from scipy.special import erfc # to integrate a Gaussian
#from scipy import optimize
import scipy.constants as const

# this is the Caltech py-controls package from Richard Murrary's group
# https://python-control.readthedocs.io/en/0.8.3/intro.html#installation
# Recommend installing in a dedicated Anaconda python environment:
# conda install -c conda-forge control
import control
import control.matlab as mat

from timeit import default_timer as timer  # this is for timing the ODE solvers
from IPython.display import Image, SVG, display

import matplotlib.pyplot as plt

import loopology
loopology.set_plot_params()

# uncomment if you have a Mac with Retina display
#%config InlineBackend.figure_format = 'retina'

# this is a nice jupyter lab theme:
# jupyter labextension install @oriolmirosa/jupyterlab_materialdarker



# params of the system
lambduh = 1064e-9  # [m] laser wavelength
w_0 = 2 *np.pi * const.c / lambduh
m_mirror = 40 # [kg] mirror mass



## Outline
1. Why Feedback?
1. Linear Systems
1. Laplace Domain
1. Feedback Loops
    1. Nomenclature
    1. How to calculate TFs for SISO
    1. MIMO calculations
        1. Adjacency Matrix
1. Noise Analysis
1. Range Analysis
1. Stability Analysis
1. Optimal Feedback

<div class="alert alert-warning">

# Why do we need Feedback Controls?
Within GW detectors, many of the systems are very weakly nonlinear, and feedback is useful to keep it linear. The interferometer itself has mainly nonlinear readbacks of the displacement degrees of freedom and so the feedback systems are used to keep it at an operating point which maximizes the sensitivity. i.e. if the laser field is not resonant in the LIGO optical cavities, LIGO doesn't work. Much like the [inverted pendulum](https://youtu.be/XWhGjxdug0o), there is a small 'capture' range where the behavior is linear.

In experimental physics, there is a lot of discussions of the virtues of making 'null' measurements (e.g. the Wheatstone Bridge in electronics, the Michelson interferometer in displacement measurements). These are special cases of using feedback to achieve the optimum operating point. The experimenter manually adjusts the operating point to make a null measurement.

### some nuggets of wisdom about control systems
1. For linear systems, the application of noiseless feedback does not increase or decrease the sensitivity of that system to external disturbances. i.e. a GW detector, operating in the linear regime, has the same sensitivity whether or not the feedback gain is zero or infinity.
1. When using feedback, we have to make sure that the design respects [**Causality**](https://en.wikipedia.org/wiki/Kramers%E2%80%93Kronig_relations). i.e. we cannot have useful negative feedback if the delay in the loop is too large. This is encapsulated in the "phase margin" of Bode plots, but can be confusing to interperet if the phase delay exceeds 180 deg.
1. Practically, the causality condition limits how much gain one can have in the loop, or how much low pass filtering: there is a correspondence between the magnitude and phase of linear systems and that includes the feedback controllers that we design.
1. Feed *forward* (or "sensor correction*) is one way to accomplish some noise suppression without being limited by large delays in the physical plant.
    
</div>

## [Linear Systems](https://en.wikipedia.org/wiki/Linear_system)
[Linear Systems](https://en.wikipedia.org/wiki/Linear_system) are key for the understanding of linear, time-invariant (LTI) control systems. 

We understand that all physical systems are nonlinear at some level, but it is useful to analyze their linear (small signal) response. Some systems (e.g. the Maxwell's Equations for E&M) are very, very linear, while others (like the Pound-Drever-Hall signal of a Fabry-Perot cavity), are very nonlinear (for a high Finesse FP cavity, the linear regime is less than 0.1% of the phase space).

### Assumptions:
1. Scaling the input by a factor $\alpha$, will also scale the output by the factor $\alpha$
1. For time-invariant linear systems (LTI), the function that relates the inputs to the outputs does not change as a function of time. Real physical systems do change, but it is often useful to study their slowly drifting parameters by assuming that we can treat the system as an LTI system at each instance during its evolution.
1. If all of the inputs to the system are zero, then all of the outputs will eventually go to zero.
1. We will usually analyze these systems in the *frequency domain*. This is a very useful techique that allows us to design the feedback in a simple way, using just algebra, rather than use integrals to compute the input-output relations.


## Laplace Domain
It is common to analyze LTI systems in the Laplace or "s" domain.

### Transfer Functions
The concept of the *Transfer Function* is a useful one in the study of control systems. This is the s-domain (or f-domain, or z-domain) representation of some system. For example the Force-to-Displacement Transfer Function of the simple pendulum is:
$$ \frac{x(\omega)}{F(\omega)} = \frac{1/m}{\omega_0^2 + i \frac{\omega_0 \omega}{Q} - \omega^2}$$
where $Q$ is the mechanical Quality factor of the pendulum, $\omega_0$ is the resonant frequency, and $m$ is the mass of the pendulum bob. This function describes the response of the pendulum bob to a sinusoidal Force, $F = F_0 e^{i \omega t}$, applied to the mirror.

## Feedback Loops
Recommend [Astrom and Murray (2009)](https://www.cds.caltech.edu/~murray/amwiki) for a nice introduction. Here lets define some terms to get started:
* **Open Loop Gain (OLG):** once around the open loop: $$OLG = P \times C \times A$$
* **Closed Loop Gain (CLG):** $$CLG = \frac{1}{1 - OLG}$$
* **Forward Loop Gain (FLG):** the open-loop gain between 2 points in the loop (e.g. between a signal injection point and the next readout port) 
* [Sensitivity](https://en.wikipedia.org/wiki/Sensitivity_(control_systems)): roughly speaking, this is the maximum peak in the CLG (i.e. how much "gain peaking" is there?)
* **Plant:** this is the physical system that we are trying to control (e.g. a the position of a pendulum, the temperature of a box, the frequency of a quartz oscillator, etc.
* **Actuator:** this is what we use to change the plant (e.g. a force, a heat source, a variable capacitor, etc.)
* **Controller:** this is where the magic happens - this is where the feedback loop transfer function is implemented. IF its digital, the number of poles & zeros here can be very large $\sim O(100)$
* **Error Signal ($\epsilon$):** the input to the feedback Controller $C(s)$
* **Control Signal ($u$):** the out of the feedback controller (e.g. the voltage that drives the actuator)
* [Dynamic Range](https://en.wikipedia.org/wiki/Dynamic_range): multiple definitions abound; one useful definition is that the dynamic range is the ration of (1) the largest possible signal a sensor can receive without 'saturating', to (2) the fundamental noise level of that sensor. For a low-noise opamp, the dynamic range is ~ (10 V)(1 nV/rHz) ~ $10^{11}$ for a 1 second measurement.

In [ ]:
SVG(filename='SISO-loop.svg')

# Example 1: Feedback Control of a Suspended Fabry-Perot Cavity
The Plant, P(s), includes the frequency response of the FP cavity:
$$ \frac{E_{\rm refl}}{x_{\rm mirror}}(\omega) = \frac{\rm DC~gain}{1 + i \omega/\omega_c}$$
as well as the pendulum motion of the mirror to an applied force:
$$ \frac{x(f)}{F(f)} = \frac{1/m}{\omega_0^2 + i \frac{\omega_0 \omega}{Q} - \omega^2}$$
where $\omega = 2 \pi f$ and $\omega_c$ is $2 \pi \times f_c$, the cavity pole frequency.

For this example, we 

In [ ]:
f = np.logspace(-1, 4, 500)
w = 2*np.pi * f

f_cav = 375 # cavity pole frequency [Hz]

# pendulum poles
f_pend = 1
p1 = 1 * np.exp(1j * 88*np.pi/180)
p_pend = -2*np.pi*np.array([p1, np.conj(p1)]) # must be complex conjugate
# look up Kramers-Kronig relations

# Convert the zpk model into state-space
P = mat.zpk2ss([], p_pend, 1/m_mirror/(2*np.pi*f_pend)**2)
Pend = control.ss(P[0], P[1], P[2], P[3])

P = mat.zpk2ss([], [-2*np.pi*f_cav], 3e12)
Cavity = control.ss(P[0], P[1], P[2], P[3])

# the feedback controller
C = mat.zpk2ss([-2*np.pi*30, -2*np.pi*30], [0, -2*np.pi*1500], 2e5)
C = control.ss(C[0], C[1], C[2], C[3])

# The Actuator, A, is flat up to 1111 Hz, with a constant force coeff [Newtons/Volt]
A = mat.zpk2ss([],[-2*np.pi*1111],1)
A = control.ss(A[0], A[1], A[2], A[3])

# multiply them together 
olg = control.series(Pend, Cavity, C, A)

# unity gain feedback connection of the OLG to make CLG
clg = control.feedback(1, olg)

#w, mag, phase = sig.bode(olg, w=w)
mag_pend,   pha_pend,   _ = control.freqresp(Pend,   omega=w)
mag_cav,   pha_cav,   _ = control.freqresp(Cavity,   omega=w)
mag_c,   pha_c,   _ = control.freqresp(C,   omega=w)
mag_olg, pha_olg, _ = control.freqresp(olg, omega=w)
mag_clg, pha_clg, _ = control.freqresp(clg, omega=w)
mag_clg = mag_clg.flatten()
mag_olg = mag_olg.flatten()
pha_clg = pha_clg.flatten()
pha_olg = pha_olg.flatten()
mag_pend = mag_pend.flatten()
pha_pend = pha_pend.flatten()
mag_cav = mag_cav.flatten()
pha_cav = pha_cav.flatten()
mag_c = mag_c.flatten()
pha_c = pha_c.flatten()
#hclg = 1 / (1 - holg)

# plot the results
fig,ax = plt.subplots(2, 2, figsize=(19,10), sharex='col')

ax[0,0].semilogx(f, 20*np.log10(mag_pend * mag_cav), label='Plant', ls='--')
ax[0,0].semilogx(f, 20*np.log10(mag_c), label='Controller', ls='--')

ax[0,0].semilogx(f, 20*np.log10(mag_clg), label='CLG')
ax[0,0].semilogx(f, 20*np.log10(mag_olg), label='OLG')
ax[0,0].set_ylim([-60, 141])

ax[0,0].set_ylabel('Mag [dB]')
ax[0,0].set_xticklabels([])
ax[0,0].legend()


ax[1,0].semilogx(f, 180/np.pi*pha_pend, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_c, ls='--')
ax[1,0].semilogx(f, 180/np.pi*pha_clg)
ax[1,0].semilogx(f, 180/np.pi*pha_olg)

ax[1,0].set_xlabel('Freq [Hz]')
ax[1,0].set_ylabel('Phase [deg]')
ax[1,0].set_ylim([-180, 181])
ax[1,0].set_yticks(np.arange(-180, 181, 60))


# now make the impulse and step response plots
T = np.linspace(0, 0.051, 1000)
t, y = control.impulse_response(clg,T)
ax[0,1].plot(t, y, label='Impulse Response')
ax[0,1].legend()

t, y = control.step_response(clg,T)
ax[1,1].plot(t, y, label='Step Response')
ax[1,1].set_xlabel('Time [s]')
ax[1,1].legend()


# Get the gain and phase margins from the Open Loop Gain
gm, pm, wg, wp = control.margin(mag_olg, 180/np.pi*pha_olg, w)
ax[0,0].axvline(wg/2/np.pi, lw=3, c='xkcd:Brown', alpha=0.3)
ax[1,0].axvline(wp/2/np.pi, lw=3, c='xkcd:Brown', alpha=0.3)
#gm, pm, wg, wp = control.margin(olg)

plt.show()
print('Phase Margin = {:0.1f} deg.'.format(pm))
print('Gain Margin  = {:0.1f} dB.'.format(20*np.log10(gm)))
#K, CL, gam, rcond = control.hinfsyn(P, 1, 1)

# I don't think the infinite feedthrough or condition number warnings are that meaningful.

## Noise Analysis
#### How to calculate the TF from the noise injection points to elsewhere in the system:
Simple - 
1. The loop suppresses the disturbance, $d$, so that right after the injection point: $$\textrm{in-loop noise} = d \times \frac{1}{1 - OLG}$$
1. From that point multiply by the forward loop gain between the injection point and the point of interest.

#### Example
* How big is the contribution of the force noise "Disturbance" in the error signal? After appling the CLG, we see that the only block between the injection of force noise and the error signal is the plant block, $P$:

$$\epsilon = d \times \frac{P}{1-OLG}$$ 

In the case of (quantum noise) for a single mirror in a Fabry-Perot cavity, we have:

$$F_{\rm RPN}(\omega) = \frac{2}{c} \sqrt{\frac{h c}{\lambda}\frac{P_{in}}{\omega_{cav}^2 + \omega^2}} $$ 

In [ ]:
# example radiation pressure noise
Pcav = 1e6 # [W] cavity power

F_RP = (2 / const.c) * np.sqrt(Pcav * const.hbar * w_0 / (1 + (w/(2*np.pi*f_cav))**2))
F_RP *= np.ones_like(w)

x_shot = 1e-20 * np.ones_like(w) * (1 + (w/(2*np.pi*f_cav)))

err = (F_RP * mag_pend)**2 + (x_shot)**2
err = np.sqrt(err)
err *= mag_clg

fig,ax = plt.subplots(1, figsize=(13,7), sharex='col')
ax.loglog(f, F_RP/m_mirror/w**2, label='RP noise')
ax.loglog(f, x_shot, label='shot noise')
ax.loglog(f, err, label='Closed-Loop Error Signal')

ax.set_ylim([1e-23, 1e-16])
ax.legend()
ax.set_xlabel(r'Frequency [Hz]')
ax.set_ylabel(r'Noise [m/$\sqrt{\rm Hz}$]')
ax.set_title('Sensitivity of the Interferometer')

plt.show()

## Range Analysis

We would like to ensure that the feedback signal we are using does not saturate the DAC nor the analog electronics. So let's compute the actuation range required for this.

We have already computed **Closed-Loop Error Signal** (above). So using our "loopology" algebra, we know how to compute the control signal spectrum:

$$ F_{ctrl}(\omega) = C(\omega) \times A(\omega) \times \epsilon(\omega)$$

So that's useful. That is the Amplitude Spectral Density ($ASD = \sqrt{PSD}$) of the **Force** applied to the mirror. To find out if we are saturating the actuator, we need to turn this into a time series quantity. We do this by integrating the [PSD](http://www.pmaweb.caltech.edu/Courses/ph136/yr2012/1206.1.K.pdf).

$$ F_{ctrl}^{RMS} = \sqrt{\int_{f_1}^{f_2} F_{ctrl}^2(\omega) d\omega}  $$

Well, that's swell. But so what? Well...now we assume that the noise processes, which the PSD represents, are non-white, [Gaussian noise](https://en.wikipedia.org/wiki/Gaussian_noise) sources. That means that amplitude of the noise, at any particular frequency, will have a Gaussian probability distribution around some mean. Since this is true for each frequency bin, we can assume (or prove it, if you like that kind of thing) that the sum of a bunch of Gaussians will also be a Gaussian. Actually, the Central Limit Theorem tells us that the sum of many, many whatever distributions will also be a Gaussian.

So we take a Gaussian with a Standard Deviation, $\sigma_F = F_{ctrl}^{RMS}$, and then ask:<br>
"For such a noise process, how often does the actuator try to exceed its maxium value?"<br>
and the answer to that is to just integrate the properly normalized Gaussian:

$$(\%~\mbox{of time we are saturating}) = 100 \times \int_{F_{max}}^{\infty} P(F) dF $$
which is just the complementary error function.

In [ ]:
fig,ax = plt.subplots(1,1)

fs = 16384 # sample rate of our digital controller

boo = np.linspace(0.01,5,1000)
yyy = erfc(boo) # complementary error function

ax.semilogy(boo, yyy)
ax.set_ylim([1e-12, 1])
ax.set_xlabel(r'$F_{max}$/$\sigma_F$')
ax.set_ylabel('Fraction of DAC samples which saturate')
ax.set_title('Actuator Saturations (f_sample = 16 kHz) assuming Gaussian noise')

ax.axhline(1/fs,       label='1 / second', c='xkcd:Red')
ax.axhline(1/fs/60,    label='1 / minute', c='xkcd:Orange')
ax.axhline(1/fs/60/60, label='1 / hour',   c='xkcd:Sea Green')
#ax.axhline(1/24/60/60)

ax.legend()
plt.show()
print(r'So, for a 16 kHz sample rate, we get a 5-$\sigma$ event approximately every {0:0.1f} days'.format(1/(fs*yyy[-1])/60/60/24))

## Stability Analysis
There are many ways to analyze the loop stability:
1. Bode plot: check gain and phase margins as in OLG plot above
1. Lyapunov Stability
1. Root-Locus Plot
1. Nyquist Plot

In the ["Gang of 4"](http://robotics.caltech.edu/wiki/images/b/be/CDS110_Week9_Lecture1.pdf) plot below, we show a common 2x2 representation of the loop for making the stability analysis:
* $S = 1 / (1 + OLG)$; this is called the closed-loop gain, and also the "Sensitivity Function"
* $T = OLG / (1 + OLG)$; the "Complementary Sensitivity Function"
* $ P \times S$; the "Load Sensitivity Function"
* $ C \times S$; the "Noise Sensitivity Function"


In [ ]:
#control.gangof4_plot(control.series(Pend, Cavity),control.series(C, A), omega=w)
#plt.show()
for k in [1.5, 2, 2.5, 3]:
    _,_,_ = control.nyquist(-olg, omega=(10**(k), 10**(k+0.5)), Plot=True)

plt.title('Nyquist Plot')
plt.show()

## Optimal Feedback
There are many ways to design feedback loops. Classicaly, some of the non-algorithmic ones are:
* PID Control: standard textbook approaches - tune P first, then I, then D. All by looking at the step function response.
* Higher-order systems: inspection of Bode Plot and addition of poles & zeros "by hand"

The most simple form of optimal feedback is known as the Linear Quadratic Regulator (LQR). This method essentially minimizes a weighted combination of the feedback loop's error signal and control signal:

$$s = \int \epsilon(t)^2 + \alpha \int u(t)^2 $$
where $\epsilon(t)$ is the error signal, $u(t)$ is the feedback control signal, and $\alpha$ is a weight that the user adjusts to choose how much each term matters. For example, in some cases (e.g. a self driving car) it is important not only to keep the system well-controlled, but to also conserve energy (i.e. minimizing $u(t)^2$). However, its clear from the formula that this method weights the overall RMS of the signal without regard to any frequency dependence. In the control systems world, there are then more sophisticated schemes to have different weighting, but in many cases where we analyze LTI systems it is more clear to produce a *Cost Function*, $s$, in the frequency domain.

### Definition of the Cost Function
Some considerations when choosing how to make an 'optimal' linear feedback controller:

1. Minimize a frequency-weighted version of the error signal: $\int W_{\epsilon}(f) \epsilon(f) df$
1. Minimize a frequency-weighted version of the control signal: $\int W_{ctrl}(f) u(f) df$
1. Minimize the rate of actuator saturations (i.e. minimize the *un-weighted* control signal)
1. Minimize the rate of sensors (ADC) saturations
1. avoid having any poles in the right half of the complex plane
1. Minimize the amount of gain peaking
1. Minimize the chance of loop instability for small, expected parameter variations.
1. Minimize the overall number of poles + zeros in the system (helps to reduce number of components in analog circuits or computation time in digital systems). It is not too helpful to shape the loop precisely for a very specific input noise spectrum unless you are sure that the spectrum will not change very much with time (true for shot noise, falso for seismic/acoustic noise).

In most practical cases, it does not make sense to make the overall cost function, $s$, depend quadratically on the cost. For example, we do not really care if we can reduce the rate of actuator saturations from 1/day to 1/month. Similarly, if the interferometer range is reduced to 1 Mpc, we do not really care if the range is further reduced to 0.5 Mpc - its all bad. So, when numerically seeking to optimize a multi-dimensional function like this, we want to make sure that the optimization manifold is well behaved, with no extremely huge mountains and no extremely deep valleys. This will allow the global optimization function to search efficiently.

### Variation of the Parameters
Once we're given the systems physical parameters and all of the noise inputs, the only thing we have left to design is the feedback controller, $C(s)$. We can think about this function in the following way:

$$ C(s) = \frac{num(s)}{den(s)}$$

where $num(s)$ and $den(s)$ are polynomials of order N. The roots of $num(s)$ are called the ***zeroes*** of the transfer function, and the roots of $den(s)$ are the ***poles***.
(you may have seen the use of poles and residues in classes on Complex Analysis or scattering problems in Quantum Mechanics, e.g. scattering of plane waves in 1D from any kind of potential well or barrier.)
To optimize the feedback system, we simply vary the values of the poles and zeros (or, equivalently, the coefficients of the polynomials) in the complex plane.

An alternative approach, called the partial fraction expansion seeks to make the optimization easier by representing $C(s)$ [as a sum of fractions](https://www.sintef.no/projectweb/vectorfitting/references/).

### Multi-Dimensional Search for optimum controller
Normal optimization methods based on stochastic gradient descent can often fail in multi-dimensional situations. The MCMC methods used for estimating the parameters of some experimental data can be used, by they are not as efficient at finding the optimum. They do however give more information about the shape of the search space.

Instead, we can use so-called Global Methods, such as Particle Swarm, Simulated Annealing, Genetic Algorithms, etc.